In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.8 MB/s eta 0:00:00


In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
os.chdir('drive/MyDrive/chatbot')

In [ ]:
import pandas as pd
data = pd.read_csv('chitchat_clean.csv')

In [ ]:
data.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
data.rename(columns = {'0':'question', '1':'answer'}, inplace = True)

In [ ]:
data.head()

,question,answer
0,در ھر صورت خیلی ممنون,خواھش میکنم
1,سلام خوبین ؟,سلام ممنونم
2,سلام کجایی ؟,سلام خونم
3,اوکی مرسی,خواھش
4,مرسی کاکو,فدات مخلصیم


In [ ]:
len(data)

851

In [ ]:
data = (data.dropna())

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
from transformers import EncoderDecoderModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-multilingual-cased', 'bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bia

In [ ]:
MAX_SOURCE_LENGTH = 8
MAX_TARGET_LENGTH = 8

BATCH_SIZE = 16

LEARNING_RATE = 2e-5
EPOCHS = 10


In [ ]:
from torch.utils.data import DataLoader, Dataset

class PersianQADataset(Dataset):

    def __init__(self, data, tokenizer, max_source_length, max_target_length):
        self.encoder_inputs = []
        self.decoder_inputs = []
        self.decoder_targets = []
        self.tokenizer = tokenizer

        for i in range(len(data)):
            source = data.iloc[i]['question']
            target = data.iloc[i]['answer']

            source_tokenized = tokenizer.encode_plus(source, max_length=max_source_length, pad_to_max_length=True)
            target_tokenized = tokenizer.encode_plus(target, max_length=max_target_length, pad_to_max_length=True)

            self.encoder_inputs.append(source_tokenized['input_ids'])
            self.decoder_inputs.append(target_tokenized['input_ids'][:-1])
            self.decoder_targets.append(target_tokenized['input_ids'][1:])

    def __len__(self):
        return len(self.encoder_inputs)

    def __getitem__(self, index):
        source_ids = self.encoder_inputs[index]
        target_ids = self.decoder_inputs[index]
        target_labels = self.decoder_targets[index]

        source_ids = torch.tensor(source_ids, dtype=torch.long)
        target_ids = torch.tensor(target_ids, dtype=torch.long)
        target_labels = torch.tensor(target_labels, dtype=torch.long)

        return {"input_ids": source_ids, "decoder_input_ids": target_ids, "decoder_attention_mask": (target_ids != tokenizer.pad_token_id), "labels": target_labels}



In [ ]:

train_dataset = PersianQADataset(train_data, tokenizer, MAX_SOURCE_LENGTH , MAX_TARGET_LENGTH)
test_dataset = PersianQADataset(test_data, tokenizer, MAX_SOURCE_LENGTH , MAX_TARGET_LENGTH)


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
import torch.optim as optim
import torch.nn as nn
import torch



def train(model, iterator, optimizer, criterion, device):
    model.train()

    epoch_loss = 0
    for batch in iterator:
        input_ids = batch["input_ids"].to(device)
        decoder_input_ids = batch["decoder_input_ids"].to(device)
        decoder_attention_mask = batch["decoder_attention_mask"].to(device)
        target_ids = batch["labels"].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask, labels=target_ids)

        loss = criterion(outputs.logits.view(-1, outputs.logits.shape[-1]), target_ids.view(-1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    torch.save(model,'model.pt')
    return epoch_loss / len(iterator)


def evaluate(model, iterator, criterion, device):
    model.eval()

    epoch_loss = 0
    with torch.no_grad():
        for batch in iterator:
            input_ids = batch["input_ids"].to(device)
            decoder_input_ids = batch["decoder_input_ids"].to(device)
            decoder_attention_mask = batch["decoder_attention_mask"].to(device)
            target_ids = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask, labels=target_ids)
            loss = criterion(outputs.logits.view(-1, outputs.logits.shape[-1]), target_ids.view(-1))

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


In [ ]:

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    train_loss = train(model, train_loader, optimizer, criterion, device)
    print(f"Train Loss: {train_loss:.3f}")
    test_loss = evaluate(model, test_loader, criterion, device)
    print(f"Test Loss: {test_loss:.3f}")

Epoch 1/10
Train Loss: 0.909
Test Loss: 1.316
Epoch 2/10
Train Loss: 0.548
Test Loss: 1.222
Epoch 3/10
Train Loss: 0.377
Test Loss: 1.175
Epoch 4/10
Train Loss: 0.330
Test Loss: 1.129
Epoch 5/10
Train Loss: 0.274
Test Loss: 1.155
Epoch 6/10
Train Loss: 0.222
Test Loss: 1.198
Epoch 7/10
Train Loss: 0.222
Test Loss: 1.210
Epoch 8/10
Train Loss: 0.208
Test Loss: 1.156
Epoch 9/10
Train Loss: 0.166
Test Loss: 1.185
Epoch 10/10
Train Loss: 0.153
Test Loss: 1.179


In [ ]:
def generate_answer(model, tokenizer, question, max_length):
    model.eval()

    input_token = tokenizer(question, max_length=max_length, truncation=True, return_tensors="pt")
    input_token = input_token.to(device)

    generated_ids = model.generate(input_token['input_ids'], decoder_start_token_id=tokenizer.cls_token_id)
    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

    return preds[0]

In [ ]:
# question = 'ممنونم خدا نگه دار'
# generate_answer(model, tokenizer, question, MAX_TARGET_LENGTH)

In [ ]:
def talk(n=5):
    for i in range(n):
        question = input("user: ")
        answer = generate_answer(model, tokenizer, question, MAX_TARGET_LENGTH)
        print("chatbot : ",answer)


In [ ]:
talk()

user: سلام چطوری ؟
chatbot :  سلام فدات
user: خوبی ؟ خوش میگذره ؟
chatbot :  اره خوبه
user: چیکار میکنی ؟
chatbot :  کار خاصی نمیکن
user: موفق باشی 
chatbot :  تو هم همینطور
user: خداحافظ
chatbot :  در پناه خدا


#Finetune on kholasenevisi commments


In [ ]:
!ls

chatbot_transbert-model.pt  Chitchat_formatted.txt  PerCQA
chitchat		    comments_cleaned	    PerCQA_clean.csv
chitchat_clean.csv	    model.pt


In [ ]:
os.chdir('comments_cleaned')

In [ ]:
!ls

kholaseNevisi.csv


In [ ]:
import pandas as pd
data = pd.read_csv('kholaseNevisi.csv')

In [ ]:
data.rename(columns = {'0':'question', '1':'answer'}, inplace = True)

In [ ]:
data.head()

,question,answer
0,سلام این برنامه جمع بندی منه و هدفم از منتشر ک...,اینطوری که آزمون ها پشت سرهم گذاشته شده یاد پش...
1,سلام این برنامه جمع بندی منه و هدفم از منتشر ک...,به نظر من که کنکورای به بعد رو فقط بزن به جاش ...
2,سلام این برنامه جمع بندی منه و هدفم از منتشر ک...,این تعداد ازمون و اونم پشت سر هم واقعا زیاده و...
3,سلام این برنامه جمع بندی منه و هدفم از منتشر ک...,به نظرم آزمونا رو کمتر کن و تایم بیشتری برای م...
4,سلام با زدن این تاپیک نه قصد جو دادن الکی دارم...,منابع یک سال یک دانش آموز درس خون و خوب رو میش...


In [ ]:
len(data)

4197

In [ ]:
data = (data.dropna())

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
%cd ..

/content/drive/MyDrive/chatbot


In [ ]:
from transformers import EncoderDecoderModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-multilingual-cased', 'bert-base-multilingual-cased')
#fine tune on previous model
model = torch.load('model.pt')

In [ ]:
MAX_SOURCE_LENGTH = 32
MAX_TARGET_LENGTH = 32

BATCH_SIZE = 32

LEARNING_RATE = 2e-5
EPOCHS = 10


In [ ]:
train_dataset = PersianQADataset(train_data, tokenizer, MAX_SOURCE_LENGTH , MAX_TARGET_LENGTH)
test_dataset = PersianQADataset(test_data, tokenizer, MAX_SOURCE_LENGTH , MAX_TARGET_LENGTH)


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [ ]:
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    train_loss = train(model, train_loader, optimizer, criterion, device)
    print(f"Train Loss: {train_loss:.3f}")
    test_loss = evaluate(model, test_loader, criterion, device)
    print(f"Test Loss: {test_loss:.3f}")

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Train Loss: 5.116
Test Loss: 3.904
Epoch 2/10
Train Loss: 3.823
Test Loss: 3.562
Epoch 3/10
Train Loss: 3.539
Test Loss: 3.372
Epoch 4/10
Train Loss: 3.359
Test Loss: 3.283
Epoch 5/10
Train Loss: 3.241
Test Loss: 3.220
Epoch 6/10
Train Loss: 3.137
Test Loss: 3.172
Epoch 7/10
Train Loss: 3.044
Test Loss: 3.134
Epoch 8/10
Train Loss: 2.951
Test Loss: 3.091
Epoch 9/10
Train Loss: 2.860
Test Loss: 3.055
Epoch 10/10
Train Loss: 2.763
Test Loss: 3.050


In [ ]:
talk()

user: سلام


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


chatbot :  سلام سلام سلام سلام سلام سلام منم سلام سلام من
user: سلام حالت خوبه ؟ چه خبر ها ؟
chatbot :  نقل قول نوشته اصلی توسط نمایش پست ها سلام دوستان من تا حدودی حدود
user: سلام دوستان دارم برنامه ریزی میکنم برای کنکو ر امسال به نظرتون از چه کتابی شروع کنم ؟
chatbot :  نقل قول نوشته اصلی توسط نمایش پست ها سلام دوستان من تازه شروع کرد
user: سلام برای  کنکور استرس دارم
chatbot :  نقل قول نوشته اصلی توسط نمایش پست ها سلام دوستان من که میخوام


KeyboardInterrupt: ignored